# Le but de ce notebook est d'identifier des caractéristiques très simples permettant de différencier des textes écrits par plusieurs auteurs
## (en l'occurrence Molière et Corneille)

In [12]:
import re
import os
import random
import math
# pour supprimer les accents
import unidecode  
# pour le stemming
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

# le répertoire de travail
directory = os.path.abspath('')

# retourne tous les fichiers txt présents dans le repertoire 'path'
def all_txt_files_in_directory(path: str):
    return [os.path.join(path,f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.txt')]

# découpe une phrase en mots, en ne gardant que les lettres
# 'il fait beau !' => ['il', 'fait', 'beau']
def split_sentence(sentence):
    words = re.findall(r"\b[\w'^\d]+\b", sentence.rstrip())
    return words
    #return [stemmer.stem(w) for w in words]
    # mot sans accents
    #return [unidecode.unidecode(w) for w in words]

# infique si la ligne 'ligne' est une ligne valide ou si elle doit être ignorée (par exemple si elle vide)
def ligne_valide(ligne: str) -> bool:
    if ligne.startswith('Scène '):
        return False
    mots = split_sentence(ligne)
    if len(mots)<= 2:
        return False # phrase trop petite
    if len(mots) >= 12:
        return False # phrase trop grande
    return True # phrase valide

# Pour tous les fichiers indiqués dans 'filenames' :
#   Pour toutes les lignes de chaque fichier :
#     Si la ligne est considérée comme valide (ni trop courte ni trop longue):
#       On l'ajoute au résultat que l'on va retourner
#     Sinon:
#       On ignore la ligne
def toutes_les_lignes_valides(filenames):
    result = set()
    for f in filenames:
        path = os.path.join(directory, f)
        with open(path) as file:
            for line in file:
                if ligne_valide(line):
                    result.add(line.rstrip())
    result = list(result)                    
    return result

# Retourne un dictionnaire mot => pourcentage, indiquant pour chaque mot le pourcentage de lignes contenant ce mot.
# 1.0 signifie que le mot est présent dans toutes les lignes, 0.0 signifie que le mot est totalement absent.
# Cela se rapproche du concept de Document Frequency (dans TF-IDF).
def calcul_pourcentage_phrases_contenant_mot(lignes) -> dict:
    nombre_de_phrases_contenant_mot = dict()
    for line in lignes:
        mots = set(split_sentence(line))
        for mot in mots:
            if mot in nombre_de_phrases_contenant_mot:
                nombre_de_phrases_contenant_mot[mot] += 1
            else:
                nombre_de_phrases_contenant_mot[mot] = 1
    result = dict()
    for mot, nombre_de_phrases in nombre_de_phrases_contenant_mot.items():
        result[mot] = nombre_de_phrases/len(lignes)
    return result

# divise le dataset d'entraînement en une partie train (avec 'train_size' %) et une partie validation
def train_validation_split(lignes, train_size:float):
    random.shuffle(lignes)
    train_lignes = lignes[:int( train_size*len(lignes))]
    validation_lignes = lignes[len(train_lignes):]
    return train_lignes, validation_lignes

# fonction de score ultra simpliste:
#    1/ poour chaque mot de la phrase 'ligne' dont on veut déterminer l'auteur, 
#       on va regarder le % de phrase de chaque auteur contenant ce mot 
#       (dans le dictionaire 'pourcentage_phrases_contenant_mot')
#       Cela ressemble beaucoup à l'idée du Document Frequency du TF-IDF
#    2/ on fait la somme de tous ces %, et on retourne cette somme
#
# Par exemple, pour la phrase: "Il fait beau":
#    1/ on regarde le % de phrases de l'auteur contenant les mots 'Il' , 'fait', 'beau' (par exemple : 10%, 2%, 3%)
#    2/ on retourne la somme de ces pourcentages, ici 0.15 (15%)
def calcul_caracteristique(ligne: str, pourcentage_phrases_contenant_mot:dict):
    score = 0
    for mot in split_sentence(ligne):
        if mot in pourcentage_phrases_contenant_mot:
            score += pourcentage_phrases_contenant_mot[mot]
    return score

# mesure la performance de la caractéristique calculée plus haut à déterminer si une ligne a été écritre par Molière (TP) 
# et on calcule la matrice de confusion associée
def calcul_matrice_de_confusion(lignes_moliere, lignes_corneille, verbose: bool = False) -> float: 
    TP = 0 # y_true = Molière ,  y_pred = Molière
    TN = 0 # y_true = Corneille, y_pred = Corneille
    FN = 0 # y_true = Molière,   y_pred = Corneille
    FP = 0 # y_true = Corneille, y_pred = Molière 
    
    # Pour toutes les lignes (du jeu de données de validation) écrites par Molière,
    # on vérifie si notre caractéristique est capable de les identifier correctement comme étant écrites par Molière (TP)
    for l in lignes_moliere:
        score_moliere = calcul_caracteristique(l, pourcentage_phrases_moliere_contenant_mot)
        score_corneille = calcul_caracteristique(l, pourcentage_phrases_corneille_contenant_mot)
        if score_moliere>score_corneille:
            if TP == 0 and verbose:
                print('Exemple de TP (Texte de Molière, bien identifié):')
                print('\t\t'+l)
                print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
            TP += 1
        else:
            if FN == 0 and verbose:
                print('Exemple de FN (Texte de Molière, mal identifié):')
                print('\t\t'+l)
                print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
            FN += 1
            
    # Pour toutes les lignes (du jeu de données de validation) écrites par Corneille,
    # on vérifie si notre caractéristique est capable de les identifier correctement comme n'étant pas écrites par Molière (TN)
    for l in lignes_corneille:
        score_moliere = calcul_caracteristique(l, pourcentage_phrases_moliere_contenant_mot)
        score_corneille = calcul_caracteristique(l, pourcentage_phrases_corneille_contenant_mot)
        if score_corneille>score_moliere:
            if TN == 0 and verbose:
                print('Exemple de TN (Texte de Corneille, bien identifié):')
                print('\t\t'+l)
                print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
            TN += 1
        else:
            if FP == 0 and verbose:
                print('Exemple de FP (Texte de Corneille, mal identifié):')
                print('\t\t'+l)
                print(f'\t\tScore Molière: {round(score_moliere,4)}, Score Corneille: {round(score_corneille,4)}')
            FP += 1
    
    if verbose:
        print(f'TP: {TP}, TN: {TN}, FN: {FN}, FP: {FP}')
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    return accuracy


In [10]:
print(f'Working directory: {directory}')

# Liste des noms de fichiers contenant les œuvres de Molière.
# Les données proviennent de:
#    https://www.ebooksgratuits.org/ebooksfrance/moliere-oeuvres_completes_1.pdf
#    https://www.ebooksgratuits.org/ebooksfrance/moliere-oeuvres_completes_2.pdf
moliere_filenames = all_txt_files_in_directory(os.path.join(directory, 'moliere'))

# Liste des noms de fichiers contenant les œuvres de Corneille.
# Les données proviennent de:
#    https://www.ebooksgratuits.com/ebooksfrance/corneille_pierre_theatre_complet_i.pdf
#    https://www.ebooksgratuits.com/ebooksfrance/corneille_pierre_theatre_complet_ii.pdf
corneille_filenames = all_txt_files_in_directory(os.path.join(directory, 'corneille'))

random.seed(42)

# on extrait toutes les lignes valides des oeuvres de Molière    
lignes_moliere = toutes_les_lignes_valides(moliere_filenames)
random.shuffle(lignes_moliere)

# on extrait toutes les lignes valides des oeuvres de Corneille
lignes_corneille = toutes_les_lignes_valides(corneille_filenames)
random.shuffle(lignes_corneille)

# les 2 datasets (Molière & Corneille) doivent avoir la même taille
taille_dataset = min(len(lignes_moliere), len(lignes_corneille))
lignes_moliere = lignes_moliere[:taille_dataset]
lignes_corneille = lignes_corneille[:taille_dataset]
       
print(f'Lignes écrites par Molière: {len(lignes_moliere)}')
print(f'Lignes écrites par Corneille: {len(lignes_corneille)}')

# on divise les données d'entraînement en train et validation
train_size = 0.9  # 90% en train, 10% en validation
train_lignes_moliere, validation_lignes_moliere = train_validation_split(lignes_moliere, train_size=train_size)
train_lignes_corneille, validation_lignes_corneille = train_validation_split(lignes_corneille, train_size=train_size)

# pour chaque mot de chaque ligne, on compte le % de lignes de chaque auteur contenant ce mot
pourcentage_phrases_moliere_contenant_mot = calcul_pourcentage_phrases_contenant_mot(train_lignes_moliere)
pourcentage_phrases_corneille_contenant_mot = calcul_pourcentage_phrases_contenant_mot(train_lignes_corneille)

accuracy = calcul_matrice_de_confusion(validation_lignes_moliere, validation_lignes_corneille, verbose=True)
print(f'Accuracy: {accuracy}') 


Working directory: C:\Projects\MathAData\Stylometrie
Lignes écrites par Molière: 26344
Lignes écrites par Corneille: 26344
Exemple de FN (Texte de Molière, mal identifié):
		Est-ce que j'écris mal ? et leur ressemblerois-je ?
		Score Molière: 0.4115, Score Corneille: 0.4327
Exemple de TP (Texte de Molière, bien identifié):
		Fort bien. Ecoutons.
		Score Molière: 0.0495, Score Corneille: 0.0325
Exemple de TN (Texte de Corneille, bien identifié):
		Que veux-tu, page ?
		Score Molière: 0.0698, Score Corneille: 0.0853
Exemple de FP (Texte de Corneille, mal identifié):
		Quand vous saurez comment il faut la gouverner.
		Score Molière: 0.3208, Score Corneille: 0.2769
TP: 1274, TN: 1945, FN: 1361, FP: 690
Accuracy: 0.6108159392789374


In [11]:
print('Accuracy pour chaque oeuvre de Moliere')
for oeuvre in moliere_filenames:
    lignes_moliere = toutes_les_lignes_valides([oeuvre])
    accuracy = calcul_matrice_de_confusion(lignes_moliere, [], verbose=False)
    print(f'Accuracy {os.path.basename(oeuvre)}: {accuracy}') 

#sorted(oeuvres_moliere.items(), key=lambda x: x[1])
print('')    
print('Accuracy pour chaque oeuvre de Corneille')
for oeuvre in corneille_filenames:
    lignes_corneille = toutes_les_lignes_valides([oeuvre])
    accuracy = calcul_matrice_de_confusion([], lignes_corneille, verbose=False)
    print(f'Accuracy {os.path.basename(oeuvre)}: {accuracy}') 


Accuracy pour chaque oeuvre de Moliere
Accuracy amphitryon.txt: 0.4555
Accuracy dom_garcie_de_navarre.txt: 0.43167701863354035
Accuracy dom_juan.txt: 0.6053811659192825
Accuracy george_dandin.txt: 0.6017441860465116
Accuracy la_comtesse_d_escarbagnas.txt: 0.6
Accuracy la_critique_de_l_ecole_des_femmes.txt: 0.6026200873362445
Accuracy la_jalousie_du_barbouille.txt: 0.5757575757575758
Accuracy la_princesse_d_elide.txt: 0.42572741194486985
Accuracy les_amants_magnifiques.txt: 0.4426877470355731
Accuracy les_facheux.txt: 0.4132882882882883
Accuracy les_femmes_savantes.txt: 0.5116772823779193
Accuracy les_fourberies_de_scapin.txt: 0.6007905138339921
Accuracy les_precieuses_ridicules.txt: 0.5441176470588235
Accuracy le_bourgeois_gentilhomme.txt: 0.603537981269511
Accuracy le_depit_amoureux.txt: 0.4568921011874032
Accuracy le_malade_imaginaire.txt: 0.5813648293963255
Accuracy le_mariage_force.txt: 0.59375
Accuracy le_medecin_malgre_lui.txt: 0.6253041362530414
Accuracy le_medecin_volant.txt: 0